# AAPL 10-K MD&A → Grounded Bullets

This notebook demonstrates end-to-end analysis of Apple's 10-K filing:
1. Search for Apple's latest 10-K filing
2. Fetch and extract MD&A section
3. Synthesize key insights with numeric grounding
4. Verify claims against external data

**Expected runtime:** < 5 minutes


In [ ]:
import requests
import json
from datetime import datetime
import time

# Configuration
API_BASE = "https://api.nocturnal.dev"
API_KEY = "your-api-key-here"  # Replace with your API key

headers = {
    "X-API-Key": API_KEY,
    "Content-Type": "application/json"
}

print(f"🚀 Starting AAPL 10-K analysis at {datetime.now()}")


## Step 1: Search for Apple's Latest 10-K Filing


In [ ]:
# Search for Apple's latest 10-K filing
search_request = {
    "ticker": "AAPL",
    "form": "10-K",
    "year_range": [2023, 2024],
    "limit": 1
}

print("🔍 Searching for AAPL 10-K filing...")
start_time = time.time()

response = requests.post(
    f"{API_BASE}/v1/finance/filings/search",
    headers=headers,
    json=search_request
)

search_duration = time.time() - start_time
print(f"✅ Search completed in {search_duration:.2f}s")

if response.status_code == 200:
    search_results = response.json()
    filings = search_results["filings"]
    
    if filings:
        filing = filings[0]
        accession = filing["accession"]
        filing_date = filing["filing_date"]
        
        print(f"📄 Found filing: {accession}")
        print(f"📅 Filing date: {filing_date}")
        print(f"🏢 Company: {filing['company_name']}")
    else:
        print("❌ No filings found")
        raise Exception("No filings found")
else:
    print(f"❌ Search failed: {response.status_code}")
    print(response.text)
    raise Exception(f"Search failed: {response.status_code}")


## Step 2: Extract MD&A Section


In [ ]:
# Extract MD&A section from the filing
extract_request = {
    "accession": accession,
    "sections": ["mda"],  # Management's Discussion and Analysis
    "tables": True
}

print("📊 Extracting MD&A section...")
start_time = time.time()

response = requests.post(
    f"{API_BASE}/v1/finance/extract",
    headers=headers,
    json=extract_request
)

extract_duration = time.time() - start_time
print(f"✅ Extraction completed in {extract_duration:.2f}s")

if response.status_code == 200:
    extract_results = response.json()
    sections = extract_results["sections"]
    tables = extract_results["tables"]
    
    mda_content = sections.get("mda", "")
    
    print(f"📝 MD&A content length: {len(mda_content)} characters")
    print(f"📊 Tables found: {len(tables)}")
    
    # Show first 500 characters of MD&A
    print("\n📖 MD&A Preview:")
    print("=" * 50)
    print(mda_content[:500] + "..." if len(mda_content) > 500 else mda_content)
    print("=" * 50)
else:
    print(f"❌ Extraction failed: {response.status_code}")
    print(response.text)
    raise Exception(f"Extraction failed: {response.status_code}")


## Step 3: Synthesize with Numeric Grounding


In [ ]:
# Define claims to verify against the MD&A
claims = [
    {
        "id": "revenue_growth",
        "metric": "revenue",
        "operator": "yoy",
        "value": 2.0,  # Expecting 2% YoY growth
        "at": "2024-01-01"
    },
    {
        "id": "profit_margin",
        "metric": "net_income_margin",
        "operator": ">",
        "value": 20.0,  # Expecting >20% profit margin
        "at": "2024-01-01"
    }
]

# Synthesize with grounding
synthesize_request = {
    "accession": accession,
    "context": {
        "company": "Apple Inc.",
        "ticker": "AAPL",
        "analysis_focus": "financial_performance"
    },
    "claims": claims,
    "grounded": True
}

print("🧠 Synthesizing with numeric grounding...")
start_time = time.time()

response = requests.post(
    f"{API_BASE}/v1/finance/synthesize",
    headers=headers,
    json=synthesize_request
)

synthesize_duration = time.time() - start_time
print(f"✅ Synthesis completed in {synthesize_duration:.2f}s")

if response.status_code == 200:
    synthesis_results = response.json()
    
    print("\n📋 Synthesis Results:")
    print("=" * 50)
    print(synthesis_results["summary"])
    print("=" * 50)
    
    print(f"\n📊 Analysis Summary:")
    print(f"- Sections analyzed: {synthesis_results['sections_analyzed']}")
    print(f"- Tables analyzed: {synthesis_results['tables_analyzed']}")
    print(f"- Claims verified: {synthesis_results['claims_verified']}")
    
elif response.status_code == 422:
    # Claims not grounded - this is expected behavior
    error_data = response.json()
    print(f"⚠️ Claims not grounded (expected): {error_data['title']}")
    print(f"📝 Detail: {error_data['detail']}")
    
    if "evidence" in error_data:
        print("\n🔍 Evidence:")
        for evidence in error_data["evidence"]:
            print(f"- {evidence['claim_id']}: {'✅' if evidence['supported'] else '❌'} {evidence.get('details', {})}")
else:
    print(f"❌ Synthesis failed: {response.status_code}")
    print(response.text)
    raise Exception(f"Synthesis failed: {response.status_code}")


## Step 4: Performance Summary


In [ ]:
# Calculate total runtime
total_duration = search_duration + extract_duration + synthesize_duration

print("\n🎯 Performance Summary:")
print("=" * 50)
print(f"🔍 Search: {search_duration:.2f}s")
print(f"📊 Extract: {extract_duration:.2f}s")
print(f"🧠 Synthesize: {synthesize_duration:.2f}s")
print(f"⏱️ Total: {total_duration:.2f}s")
print("=" * 50)

# Check if we met the 5-minute target
if total_duration < 300:  # 5 minutes
    print("✅ SUCCESS: Analysis completed in < 5 minutes")
else:
    print("⚠️ WARNING: Analysis took > 5 minutes")

print(f"\n🏁 Analysis completed at {datetime.now()}")


## Expected Results

This notebook should:

1. **Find Apple's latest 10-K filing** in < 2 seconds
2. **Extract MD&A section** in < 10 seconds
3. **Synthesize with grounding** in < 30 seconds
4. **Return 422 error** for ungrounded claims (expected behavior)
5. **Complete end-to-end** in < 5 minutes

The 422 error for ungrounded claims demonstrates that the system correctly rejects unverified numeric claims, which is the core value proposition of the FinSight API.
